<a href="https://colab.research.google.com/github/ramnarayan-code/llm_reflection/blob/main/langgraph_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture --no-stderr
%pip install -U langchain langgraph langchain_openai langchain_experimental langsmith pandas

In [83]:
# Ref: https://medium.com/@anuragmishra_27746/future-of-coding-multi-agent-llm-framework-using-langgraph-092da9493663

In [85]:
import os
import operator
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple

from langchain_core.pydantic_v1 import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import HumanMessage
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph

import os
os.environ['OPENAI_API_KEY'] = ''

llm=ChatOpenAI()

In [86]:
class ProductOwner(BaseModel):
    user_stories: str = Field(
        description="PO defined user stories"
    )

product_owner_prompt = ChatPromptTemplate.from_template(
    '''
    **Role**: You are a product owner. You need to create user stories
**Task**: As a product owner, you are required to create 5 user stories based on the use case

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the use case
2. **Define user story and assign priority**: define user story in the below format following the agile standards
*FORMAT*
User story:

*USE CASE*
{use_case}

'''
)
product_owner_agent = create_structured_output_runnable(
    ProductOwner, llm, product_owner_prompt
)


In [102]:
class Developer(BaseModel):
    dev_team_proposal: str = Field(
        description="Developer proposal for the use case implementation"
    )

developer_prompt = ChatPromptTemplate.from_template(
    '''
    **Role**: You are a developer. You need to assess the user stories
**Task**: As a developer, you need to assess the user stories and provide the technical proposal

**Instructions**:
1. **Understand and Clarify**: Make sure you understand the use case and its related user stories
2. **Design the system**: Design a system in the below output format for the use case defining the architecture pattern, defining the components, define the potential technology stack and estimate the efforts
*FORMAT*
Architecture pattern:
Components:
Tech Stack:
ETA:


*USE CASE*
{use_case}

*USE STORIES*
{po_defined_user_stories}
'''
)
developer_agent = create_structured_output_runnable(
    Developer, llm, developer_prompt
)


In [103]:
class AgentState(TypedDict):
    use_case: str
    po_defined_user_stories: str
    dev_team_proposal: str

In [104]:
def product_owner(state):
    print(f'Product Owner agent:')
    use_case = state['use_case']
    po_response = product_owner_agent.invoke({'use_case':use_case})
    return {'po_defined_user_stories':po_response.user_stories}

def developer(state):
    print(f'Developer agent:')
    po_defined_user_stories = state['po_defined_user_stories']
    use_case = state['use_case']
    developer_response = developer_agent.invoke({'po_defined_user_stories':po_defined_user_stories, 'use_case':use_case})
    return {'dev_team_proposal':developer_response.dev_team_proposal}

In [105]:
workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("product_owner", product_owner)
workflow.add_node("developer", developer)

# Build graph
workflow.set_entry_point("product_owner")
workflow.add_edge("product_owner", "developer")
workflow.add_edge("developer", END)

app = workflow.compile()


In [106]:
for stream_msg in app.stream(
    {"use_case": [HumanMessage(content="Design a CRM system using AI")]},
    {"recursion_limit": 100},
):
    if "__end__" not in stream_msg:
        if "product_owner" in stream_msg:
          po_defined_user_stories = stream_msg["product_owner"]["po_defined_user_stories"]
          print(po_defined_user_stories)
        elif "developer" in stream_msg:
          dev_team_proposal = stream_msg["developer"]["dev_team_proposal"]
          print(dev_team_proposal)
        print("----")

Product Owner agent:
User Story 1:

As a sales representative, I want the CRM system to suggest potential leads based on AI analysis so that I can focus on high-quality prospects.

User Story 2:

As a customer support agent, I want the CRM system to provide insights on customer behavior using AI to enhance personalized interactions.

User Story 3:

As a marketing manager, I want the CRM system to automate campaign optimization through AI algorithms for better targeting and engagement.

User Story 4:

As a data analyst, I want the CRM system to generate predictive analytics reports using AI to improve decision-making processes.

User Story 5:

As a system administrator, I want the CRM system to have AI-powered security features for data protection and privacy compliance.
----
Developer agent:
**Technical Proposal for CRM System Using AI**

**Architecture pattern:**

The proposed architecture for the CRM system using AI will follow a microservices architecture pattern. This will allow fo